In [1]:
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from keras import optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten, GRU
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, to_categorical
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
import pickle
import re
import operator
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend.
/Users/aaronfrederick/Desktop/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
corpus = pickle.load(open('corpus.pkl', 'rb'))
cleaned = [re.sub('[^a-zA-Z ]', '', song) for song in corpus]

cleaned = list(set(cleaned))
tokens = [song.split() for song in cleaned]

words = []
for song in cleaned:
    for word in song.split():
        words.append(word)

lines = []
for i in range(len(words)-7):
    lines.append(words[i:i+7])

In [3]:
# lines_unique = []
# for line in tqdm(lines):
#     if line not in lines_unique:
#         lines_unique.append(line)

# len(lines_unique)

In [4]:
# pickle.dump(lines_unique, open('unique_lines.pkl', 'wb'))

In [5]:
lines_unique = pickle.load(open('unique_lines.pkl', 'rb'))

In [6]:
#Turn words to ints
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines_unique)
sequences = tokenizer.texts_to_sequences(lines_unique)
vocab_size = len(tokenizer.word_index) + 1

sequences = np.array(sequences)

In [7]:
sequences.shape

(143265, 7)

In [8]:
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)

In [9]:
#Create model *DONT USE DROPOUT - DEPENDENCY ERROR*
model = Sequential()
model.add(Embedding(vocab_size, output_dim=50, input_length=seq_length))
model.add(GRU(256, return_sequences=True))
model.add(GRU(256))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [10]:
sgd = optimizers.SGD(lr=0.1, decay=0, momentum=0.05, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 50)             270250    
_________________________________________________________________
gru_1 (GRU)                  (None, 6, 256)            235776    
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               393984    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               25700     
_________________________________________________________________
dense_2 (Dense)              (None, 5405)              545905    
Total params: 1,471,615
Trainable params: 1,471,615
Non-trainable params: 0
_________________________________________________________________


In [29]:
filepath = "weights.{epoch:02d}-{loss:.2f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=False, mode="min", period = 2)
callbacks_list = [checkpoint]

model.fit(X, y, batch_size=256, epochs=6)

Epoch 1/6
143265/143265 [==============================] - 99s 693us/step - loss: 4.5775 - acc: 0.1529
Epoch 2/6
143265/143265 [==============================] - 94s 655us/step - loss: 4.5568 - acc: 0.1556
Epoch 3/6
 28416/143265 [====>.........................] - ETA: 1:14 - loss: 4.5260 - acc: 0.1575

KeyboardInterrupt: 

In [12]:
def gen_sequence(seed, model=model, tokenizer=tokenizer, length=6, n_words=70):
    result = []
    in_word = seed
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_word])[0]
        encoded = pad_sequences([encoded], maxlen=length, truncating='pre')
        pred = model.predict_classes(encoded)
        out = ''
        for word, index in tokenizer.word_index.items():
            if index==pred:
                out = word
                break
        in_word += ' ' + out
        result.append(out)
    return ' '.join(result)

In [30]:
print(gen_sequence('me', n_words=10))
print(gen_sequence('why', n_words=10))
print(gen_sequence('we', n_words=10))
print(gen_sequence('us', n_words=10))
print(gen_sequence('tell', n_words=10))
print(gen_sequence('trust', n_words=10))

and me i know that you are my girl you
you know i know that you are my girl you
go and we were young we were young we were
and the way you know that you know what you
me what you want to be the one you know
up the way you know the way you know that
